In [1]:

import pandas as pd
import numpy as np


# Exercício 17 (Capítulo 7, RUGGIERO)

Considere a integral

$
    I = \int_{0}^{1}{e^{-x^2}dx}
$

a) Estime $I$ pela Regra de Simpsons usando $h = 0.25$

b) Estime $I$ pela Quadratura Gaussiana com 2 pontos

c) Sabendo que o valor exato de $I$ (usando 5 casas decimais) é $0.74682$, pede-se:
    
c_1) Compare as estimativas obtidas em (a) e (b);

c_2) quantos pontos seriam necessários para que a regra dos Trapézios obtivesse a mesma precisão que a estimativa obtida para $I$ em (b)?
 


## Implementação algoritmica

In [78]:
def integrar_simpson(f: 'function', 
                     h: float, 
                     a: float, 
                     b: float, 
                     verbose: bool = True
                     ) -> float:

    m = int(np.ceil((b - a) / h))
    x = np.linspace(a, b, m + 1, dtype=np.float32)

    x_i_odd = x[range(2, m, 2)]
    x_i_even = x[range(1, m, 2)]

    y = (h / 3) * (f(x[0]) + np.sum(2 * f(x_i_odd)) + np.sum(4 * f(x_i_even)) + f(x[-1]))
    err = (m * h**5) / 180 * (np.max(f(x)))

    if verbose:
        print(f'Limitante de erro = |{err}|')

    return y

In [70]:
def integrar_quadratura_gaussiana(f: 'function', 
                                  a: float, 
                                  b: float, 
                                  n: int,
                                  verbose: bool = True
                                  ) -> float:
    
    x, w = np.polynomial.legendre.leggauss(n)

    _x = 0.5 * (x * (b - a) + (b + a))
    _w = 0.5 * (w * (b - a))

    y = sum(w_i * f(x_i) for x_i, w_i in zip(_x, _w))

    return y

### Teste com exemplo 2 do capítulo 7

In [69]:

f = lambda x: np.exp(x)
h = 1/10
a = 0
b = 1

integrar_simpson(f, h, a, b)

# Deveria resultar 1.71828278
# Limitante |E| <= |1.51015e-06|


Limitante de erro = |1.5101567214514944e-06|


np.float32(1.7182829)

### Teste com exemplo 3 do capítulo 7

In [74]:

f = lambda x: np.exp(-x)
a = 0
b = 10
n = 2


integrar_quadratura_gaussiana(f, a, b, n)

# Deveria resultar 0.606102
# Erro apenas sabendo valor analitico da funcao

np.float64(0.6061021668165502)

## Resolução

In [77]:

f = lambda x: np.exp(-x**2)
h = 0.25
a = 0
b = 1


#### a) Estime $I$ pela Regra de Simpsons usando $h = 0.25$

In [91]:
I_a = integrar_simpson(f, h, a, b)
print(f'I_a = {I_a}')


Limitante de erro = |2.1701389414374717e-05|
I_a = 0.7468554377555847



#### b) Estime $I$ pela Quadratura Gaussiana com 2 pontos


In [92]:
I_b = integrar_quadratura_gaussiana(f, a, b, 2)
print(f'I_b = {I_b}')


I_b = 0.7465946882828597


#### c) Sabendo que o valor exato de $I$ (usando 5 casas decimais) é $0.74682$, pede-se:

In [93]:
I = 0.74682


#### $c_1$) Compare as estimativas obtidas em (a) e (b);

In [105]:

# Utilizando erro relativo

print(f'Erro de b é menor que de a? {I_b < I_a}')
print(f'Erro relativo de I_a {I_a / I - 1}')
print(f'Erro relativo de I_b {I_b / I - 1}')

err_I_b = I_b - I
print(f'Erro absoluto de I_b = {err_I_b}')

Erro de b é menor que de a? True
Erro relativo de I_a 4.744529724121094e-05
Erro relativo de I_b -0.00030169480884334465
Erro absoluto de I_b = -0.00022531171714035292


#### $C_2$) quantos pontos seriam necessários para que a regra dos Trapézios obtivesse a mesma precisão que a estimativa obtida para $I$ em (b)?
 


Sabe-se que

$
    |E_{TR}| = \dfrac{mh^3M_2}{12}
$

sendo 

$
    M_2 = \text{máx}_{x \in [a, b]}|f''(x)|
$

Como $m = \dfrac{b - a}{h}$, obtém-se:

$
    |E_{TR}| = \dfrac{b-a}{12} h^2M_2
$

---

Como em $I_b$ obteve-se $E_b = -0.22531171714035292 * 10^{-3}$

Para que $I_b = E_{TR}$, tem-se que:

$
    |E_{b}| = \dfrac{b-a}{12} h^2M_2
$

Deve-se estimar $M_2$:


In [111]:

# f(x) = e^{-x^2}
# f'(x) = -2x e^{-x^2}
# f''(x) = -2e^{-x^2} + 4x^2 e^{-x^2}

# Estimando f''(x) em 10.000 pontos
pontos = np.linspace(a, b, 10_000)
fdd = lambda x: -2 * np.exp(-x**2) + 4 * x**2 * np.exp(-x**2)

# Calculando M_2
M_2 = np.max(np.abs(fdd(pontos)))

print(M_2)


2.0



Considerando $b = 1$, $a = 0$ e $M = 2$, é possível avaliar que

$
    |0.22531171714035292 * 10^{-3}| \le \dfrac{(1) - (0)}{12} (h)^2 (2)
$

$
    |0.22531171714035292 * 10^{-3}| \le \dfrac{1}{12} (2h^2)
$

$
    |0.22531171714035292 * 10^{-3}| \le \dfrac{h^2}{6}
$


$
    (6)|0.22531171714035292 * 10^{-3}| \le h^2
$

$
    |1.3518703028421175 * 10^{-3}| \le h^2
$


$
    \sqrt{|1.3518703028421175 * 10^{-3}|} \le h
$


$
    h \ge 1.162699575488921 * 10^{-3}
$

$
    h \ge 0.001162699575488921
$

Considerando que o menor valor para $h$ seja $0.001162699575488921$, então

$
    m = \dfrac{1}{0.001162699575488921}
$


$
    m = \dfrac{1}{0.001162699575488921}
$


$
    m \approx 860
$

Isto é, seriam necessários mais de 860 pontos.